In [1]:
import numpy as mp
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import socket
import import_ipynb
from Model_Owner import Decision_Tree_Model_User as mu


importing Jupyter notebook from Model_Owner.ipynb


In [2]:
OWNERS = 3
TRAINING_DATASET = 'iris.csv'
DELIMITER = ','
SAVEFILE = 'iris_data.sav'


# SETUP PHASE

## 1 - TRAINING OF DECISION TREE BY DIFFERENT USERS

### A - GENERATE SET OF MODEL OWNERS

In [3]:
all_models = []
for i in range(OWNERS):
    client = 1
    r_model = mu(TRAINING_DATASET,DELIMITER,SAVEFILE)
    r_model.save_model()
    r_model.load_model()
    model = r_model.dec_model
    all_models.append(model)

#print (all_models)
    

SyntaxError: invalid syntax (<ipython-input-3-ec733bdd3ab3>, line 3)